In [4]:
from datetime import date, datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
from demeter import TokenInfo, Actuator, Strategy, RowData, ChainType, MarketInfo, AtTimeTrigger, PeriodTrigger, simple_moving_average, realized_volatility
from demeter.uniswap import UniV3Pool, UniLpMarket
from demeter.result import performance_metrics, round_results
import math

# Tokens and pool setup
usdc = TokenInfo(name="usdc", decimal=6)  # declare token usdc
eth = TokenInfo(name="eth", decimal=18)  # declare token eth
pool = UniV3Pool(token0=usdc, token1=eth, fee=0.05, quote_token=usdc)
market_key = MarketInfo("market1")
market = UniLpMarket(market_key, pool)  # declare the market

# Data path and loading configuration
market.data_path = "../data"  # Adjust if necessary
market.load_data(
    chain=ChainType.ethereum.name, 
    contract_addr="0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",
    start_date=date(2024, 4, 1),
    end_date=date(2024, 9, 30)
)

# Base Actuator configuration
def initialize_actuator(strategy_class, *strategy_args):
    actuator = Actuator()
    broker = actuator.broker
    broker.add_market(market)
    broker.set_balance(usdc, 36438.373)
    broker.set_balance(eth, 10)
    
    actuator.strategy = strategy_class(*strategy_args)
    actuator.set_price(market.get_price_from_data())
    return actuator

# Strategy classes
class NoProvisionStrategy(Strategy):
    def initialize(self):
        new_trigger = AtTimeTrigger(time=datetime(2024, 4, 1, 0, 0, 0), do=self.work)
        self.triggers.append(new_trigger)
    def work(self, row_data: RowData):
        pass

class UniswapV2Strategy(Strategy):
    def initialize(self):
        new_trigger = AtTimeTrigger(time=datetime(2024, 4, 1, 0, 0, 0), do=self.work)
        self.triggers.append(new_trigger)
    def work(self, row_data: RowData):
        lp_market: UniLpMarket = self.markets[market_key]
        lp_market.add_liquidity_by_tick(-887272, 887272)

class ConstantIntervalStrategy(Strategy):
    def __init__(self, a=100):
        super().__init__()
        self.a = a
    def initialize(self):
        market: UniLpMarket = self.markets[market_key]
        init_price = market.market_status.data.price
        market.even_rebalance(init_price)
        market.add_liquidity(init_price - self.a, init_price + self.a)

class IntervalsAroundtheCurrentPriceStrategy(Strategy):
    def __init__(self, a=10, update_interval=timedelta(days=1)):
        super().__init__()
        self.a = a
    def initialize(self):
        lp_market: UniLpMarket = self.broker.markets[market_key]
        current_price = lp_market.market_status.data.price
        lp_market.add_liquidity(current_price - self.a, current_price + self.a)
        self.triggers.append(PeriodTrigger(time_delta=timedelta(days=1), do=self.work))
    def work(self, row_data: RowData):
        lp_market: UniLpMarket = self.broker.markets[market_key]
        current_price = row_data.prices[eth.name]
        if len(lp_market.positions) > 0:
            lp_market.remove_all_liquidity()
            lp_market.even_rebalance(current_price)
        lp_market.add_liquidity(current_price - self.a, current_price + self.a)

class TwoIntervalsAroundtheCurrentPriceStrategy(Strategy):
    def __init__(self, a=10, b=1, update_interval=timedelta(days=1)):
        super().__init__()
        self.a = a
        self.b = b
    def initialize(self):
        lp_market: UniLpMarket = self.broker.markets[market_key]
        init_price = lp_market.market_status.data.price
        lp_market.add_liquidity(init_price - self.a, init_price + self.a)
        if self.broker.assets[market.base_token].balance > 0:
            lp_market.add_liquidity(init_price - self.b, init_price)
        else:
            lp_market.add_liquidity(init_price, init_price + self.b)
        self.triggers.append(PeriodTrigger(time_delta=timedelta(days=1), do=self.work))
    def work(self, row_data: RowData):
        lp_market: UniLpMarket = self.broker.markets[market_key]
        if len(lp_market.positions) > 0:
            lp_market.remove_all_liquidity()
            lp_market.even_rebalance(row_data.prices[eth.name])
        if self.broker.assets[market.base_token].balance > 0:
            lp_market.add_liquidity(row_data.prices[eth.name] - self.b, row_data.prices[eth.name])
        else:
            lp_market.add_liquidity(row_data.prices[eth.name], row_data.prices[eth.name] + self.b)

class FillUpStrategy(Strategy):
    def __init__(self, a=10):
        super().__init__()
        self.a = a
    def initialize(self):
        lp_market: UniLpMarket = self.broker.markets[market_key]
        init_price = lp_market.market_status.data.price
        lp_market.even_rebalance(init_price)
        lp_market.add_liquidity(init_price - self.a, init_price + self.a)
        self.triggers.append(PeriodTrigger(time_delta=timedelta(days=1), do=self.work))
    def work(self, row_data: RowData):
        lp_market: UniLpMarket = self.broker.markets[market_key]
        if len(lp_market.positions) > 0:
            lp_market.remove_all_liquidity()
            lp_market.even_rebalance(row_data.prices[eth.name])
        lp_market.add_liquidity(row_data.prices[eth.name] - self.a, row_data.prices[eth.name] + self.a)

class MovingAverageStrategy(Strategy):
    def initialize(self):
        self.add_column(market_key, "sma", simple_moving_average(self.data[market_key].price, timedelta(days=1)))
        self.triggers.append(PeriodTrigger(time_delta=timedelta(days=1), trigger_immediately=True, do=self.rebalance))
    def rebalance(self, row_data: RowData):
        self.markets[market_key].even_rebalance(row_data.market_status[market_key].price)

class AddByVolatilityStrategy(Strategy):
    def initialize(self):
        self.add_column(market_key, "sma_1_day", simple_moving_average(self.data[market_key].price, timedelta(days=1)))
        self.add_column(market_key, "volatility", realized_volatility(self.data[market_key].price, timedelta(days=1), timedelta(days=1)))
        self.triggers.append(PeriodTrigger(time_delta=timedelta(days=1), trigger_immediately=True, do=self.work))
    def work(self, row_data: RowData):
        lp_market: UniLpMarket = self.broker.markets[market_key]
        lp_row_data = row_data.market_status[market_key]
        if len(lp_market.positions) > 0:
            lp_market.remove_all_liquidity()
            lp_market.even_rebalance(row_data.prices[eth.name])
        if math.isnan(lp_row_data.volatility):
            return
        limit = 2 * float(row_data.prices[eth.name]) * lp_row_data.volatility
        lp_market.add_liquidity(lp_row_data.sma_1_day - limit, lp_row_data.sma_1_day + limit)

# List of strategies and actuators
strategies = [
    (NoProvisionStrategy, []),
    (UniswapV2Strategy, []),
    (ConstantIntervalStrategy, [100]),
    (IntervalsAroundtheCurrentPriceStrategy, [1000]),
    (TwoIntervalsAroundtheCurrentPriceStrategy, [1000, 500]),
    (FillUpStrategy, [1000]),
    (MovingAverageStrategy, []),
    (AddByVolatilityStrategy, [])
]
actuators = [initialize_actuator(strategy, *args) for strategy, args in strategies]
strategy_names = [
    "NoProvisionStrategy", "UniswapV2Strategy", "ConstantIntervalStrategy",
    "IntervalsAroundtheCurrentPriceStrategy", "TwoIntervalsAroundtheCurrentPriceStrategy",
    "FillUpStrategy", "MovingAverageStrategy", "AddByVolatilityStrategy"
]

# Run each strategy and collect results
results = {}
for actuator, name in zip(actuators, strategy_names):
    actuator.broker.set_balance(usdc, 36438.373)
    actuator.broker.set_balance(eth, 10)
    actuator.run()
    results[name] = actuator.account_status_df["net_value"]

# Plotting
plt.figure(figsize=(14, 8))
for name, net_value in results.items():
    plt.plot(net_value.index, net_value, label=name)
plt.xlabel("Date")
plt.ylabel("Net Value")
plt.title("Net Value of Different Trading Strategies Over Time")
plt.legend()
plt.show()


2024-10-25 15:38:04,394 - INFO - start load files from 2024-04-01 to 2024-09-30...
2024-10-25 15:38:09,637 - INFO - load file complete, preparing...
2024-10-25 15:38:16,408 - INFO - data has been prepared
2024-10-25 15:38:20,806 - INFO - Qute token is USDC
2024-10-25 15:38:20,808 - INFO - init strategy...
2024-10-25 15:38:20,813 - INFO - start main loop...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 263520/263520 [01:41<00:00, 2594.17it/s]
2024-10-25 15:40:02,405 - INFO - main loop finished
2024-10-25 15:40:08,031 - INFO - Print actuator summary


Final account status                              
Token balance in broker       
USDC      :36438.373                ETH       :10                       
Position value in markets     
market1(UniLpMarket)
token0    :USDC                     token1    :ETH                      fee(%)    :0.0500                   quote token:USDC                     
positions 
Empty DataFrame


Quote by: USDC
Account balance history                           


2024-10-25 15:40:08,810 - INFO - Backtesting finished, execute time 108.01947855949402s
2024-10-25 15:40:08,843 - INFO - Qute token is USDC
2024-10-25 15:40:08,844 - INFO - init strategy...
2024-10-25 15:40:08,858 - INFO - start main loop...


l1                  net_value    tokens       market1                   \
l2                                 USDC ETH net_value base_uncollected   
2024-04-01 00:00:00 72876.746 36438.373  10         0                0   
2024-04-01 00:01:00 72865.817 36438.373  10         0                0   
2024-04-01 00:02:00 72840.328 36438.373  10         0                0   
2024-04-01 00:03:00 72847.609 36438.373  10         0                0   
2024-04-01 00:04:00 72854.891 36438.373  10         0                0   
...                       ...       ...  ..       ...              ...   
2024-09-30 23:55:00 62436.885 36438.373  10         0                0   
2024-09-30 23:56:00 62462.895 36438.373  10         0                0   
2024-09-30 23:57:00 62462.895 36438.373  10         0                0   
2024-09-30 23:58:00 62465.498 36438.373  10         0                0   
2024-09-30 23:59:00 62465.498 36438.373  10         0                0   

l1                                   

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 263520/263520 [01:32<00:00, 2834.94it/s]
2024-10-25 15:41:41,829 - INFO - main loop finished
2024-10-25 15:41:48,033 - INFO - Print actuator summary


Final account status                              
Token balance in broker       
USDC      :36438.373                ETH       :10                       
Position value in markets     
market1(UniLpMarket)
token0    :USDC                     token1    :ETH                      fee(%)    :0.0500                   quote token:USDC                     
positions 
   lower_tick  upper_tick   pending0    pending1        liquidity
0     -887270      887270  760.50848  0.26798981  603642053772538

Quote by: USDC
Account balance history                           


2024-10-25 15:41:49,489 - INFO - Backtesting finished, execute time 100.65582537651062s
2024-10-25 15:41:49,516 - INFO - Qute token is USDC
2024-10-25 15:41:49,517 - INFO - init strategy...
2024-10-25 15:41:49,531 - INFO - start main loop...


l1                  net_value    tokens       market1                   \
l2                                 USDC ETH net_value base_uncollected   
2024-04-01 00:00:00  145753.5 36438.373  10 72876.754    2.0353984e-06   
2024-04-01 00:01:00 145731.64 36438.373  10 72865.827    2.9426024e-06   
2024-04-01 00:02:00 145680.66 36438.373  10 72840.336    4.6597311e-06   
2024-04-01 00:03:00 145695.23 36438.373  10 72847.625    5.1044055e-06   
2024-04-01 00:04:00 145709.81 36438.373  10 72854.915    5.5786792e-06   
...                       ...       ...  ..       ...              ...   
2024-09-30 23:55:00 125452.01 36438.373  10 63015.124       0.26798979   
2024-09-30 23:56:00 125509.51 36438.373  10 63046.614       0.26798979   
2024-09-30 23:57:00 125509.51 36438.373  10 63046.614        0.2679898   
2024-09-30 23:58:00 125515.26 36438.373  10 63049.764        0.2679898   
2024-09-30 23:59:00 125515.26 36438.373  10 63049.764       0.26798981   

l1                                   

 26%|███████████████████████████▏                                                                            | 68884/263520 [00:32<01:30, 2141.50it/s]


KeyboardInterrupt: 